<a href="https://colab.research.google.com/github/el-eshaano/DL-Lab/blob/main/Week-08/CharGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-03-11 18:52:59--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.008s  

2024-03-11 18:52:59 (138 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader

from tqdm.notebook import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

In [14]:
with open('./input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

unique_chars = sorted(list(set(text)))
vocab_size = len(unique_chars)

stoi = { ch:i for i,ch in enumerate(unique_chars) }
itos = { i:ch for i,ch in enumerate(unique_chars) }

tokens = torch.Tensor([stoi[x] for x in list(text)][:5000])

train_split = 0.9
n = int(train_split * len(tokens))
train, test = tokens[:n], tokens[n:]

In [15]:
class TokensDataset(Dataset):

    def __init__(self, series, seq_length=10, device="cpu"):
        self.series = series
        self.seq_length = seq_length

    def __len__(self):
        return len(self.series) - self.seq_length - 1

    def __getitem__(self, idx):
        X = self.series[idx:idx+self.seq_length].reshape(self.seq_length, -1).to(device)
        y = self.series[idx+self.seq_length].to(device)

        return X, y

In [16]:
train_dataset = TokensDataset(train, seq_length=10, device=device)
test_dataset = TokensDataset(test, seq_length=10, device=device)

In [17]:
train_data = DataLoader(train_dataset, batch_size=1024, shuffle=True)
test_data = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [18]:
class RNN(nn.Module):

    def __init__(self, input_size, hidden_size):

        super(RNN, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True) # (batch, time, input_size) -> (batch, time, hidden_size)
        self.fc = nn.Sequential(
            nn.ReLU(),
            nn.Linear(in_features=hidden_size, out_features=1), # out_features = 1 to predict the next value
        )

    def forward(self, x):
        output, status = self.rnn(x)
        output = output[:, -1, :] # (batch, time, hidden_size) -> (batch, 1, hidden_size). We take only the final output of the RNN
        output = self.fc(output)

        return output

In [19]:
input_size = 1
hidden_size = 5
learning_rate = 1e-4
num_epochs = 15
# ----

model = RNN(input_size, hidden_size).to(device)

criterion = nn.MSELoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [20]:
for epoch in tqdm(range(num_epochs), desc="Epochs"):
    for batch, targets in tqdm(train_data, desc="Batches", leave=False):

        outs = model(batch).reshape(-1)

        optimizer.zero_grad()
        loss = criterion(outs, targets)
        loss.backward()
        optimizer.step()

    print(f"After epoch {epoch}, loss={loss.item()}")

Epochs:   0%|          | 0/15 [00:00<?, ?it/s]

Batches:   0%|          | 0/5 [00:00<?, ?it/s]

After epoch 0, loss=1844.2989501953125


Batches:   0%|          | 0/5 [00:00<?, ?it/s]

After epoch 1, loss=1914.3670654296875


...

After epoch 12, loss=1858.0006103515625


Batches:   0%|          | 0/5 [00:00<?, ?it/s]

After epoch 13, loss=1924.8201904296875


Batches:   0%|          | 0/5 [00:00<?, ?it/s]

After epoch 14, loss=1880.751220703125
